<a href="https://colab.research.google.com/github/ns01988/ClientApp/blob/master/GraphRAG_IMDb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
harshitshankhdhar_imdb_dataset_of_top_1000_movies_and_tv_shows_path = kagglehub.dataset_download('harshitshankhdhar/imdb-dataset-of-top-1000-movies-and-tv-shows')

print('Data source import complete.')


100%|██████████| 175k/175k [00:00<00:00, 30.0MB/s]

Extracting files...
Data source import complete.


In [ ]:
!pip install neo4j transformers openai langchain langchain_community -q

In [ ]:
import pandas as pd
import numpy as np
from neo4j import GraphDatabase
from transformers import BertTokenizer, BertModel
from langchain_community.graphs import Neo4jGraph
from langchain_community.graphs.graph_document import GraphDocument,Node,Relationship
from langchain_core.documents import Document

In [ ]:
movies_ds = pd.read_csv("/kaggle/input/imdb-dataset-of-top-1000-movies-and-tv-shows/imdb_top_1000.csv")
movies_ds.head()

### Extract all the genres

In [ ]:
len(movies_ds)

In [ ]:
all_genres = set()
for genre_list in movies_ds["Genre"]:
    for e in genre_list.split(","):
        all_genres.add(e.strip())

In [ ]:
all_genres

## Embeddings models OPEN SOURCE
Existen varios modelos de *embedding* gratuitos que son ampliamente utilizados y reconocidos por su rendimiento en tareas de procesamiento de lenguaje natural (NLP). Aquí tienes una lista de modelos recomendados:

### 1. **Word2Vec**
   - **Descripción**: Modelo de *embedding* creado por Google que transforma palabras en vectores numéricos basados en su contexto en grandes corpus de texto.
   - **Fuente**: [Google Word2Vec pre-trained model](https://code.google.com/archive/p/word2vec/).
   - **Ventajas**: Buena representación semántica y sintáctica de palabras.
   - **Desventajas**: No maneja palabras fuera de vocabulario (*out-of-vocabulary*).

### 2. **GloVe (Global Vectors for Word Representation)**
   - **Descripción**: Desarrollado por Stanford, es un modelo que combina enfoques basados en coocurrencias para generar *embeddings* de palabras.
   - **Fuente**: [GloVe pre-trained vectors](https://nlp.stanford.edu/projects/glove/).
   - **Ventajas**: Buen balance entre representaciones locales y globales.
   - **Desventajas**: No maneja palabras fuera de vocabulario.

### 3. **FastText**
   - **Descripción**: Desarrollado por Facebook, es una extensión de Word2Vec que maneja palabras fuera de vocabulario al dividir palabras en sub-palabras.
   - **Fuente**: [FastText pre-trained models](https://fasttext.cc/docs/en/crawl-vectors.html).
   - **Ventajas**: Capaz de manejar palabras desconocidas gracias a la representación por sub-palabras.
   - **Desventajas**: Mayor tamaño en los modelos preentrenados.

### 4. **Transformer-based Embeddings (BERT y derivados)**
   - **Descripción**: BERT (Bidirectional Encoder Representations from Transformers) y otros modelos basados en Transformers como DistilBERT, RoBERTa, y ALBERT son más avanzados y consideran el contexto de las palabras bidireccionalmente.
   - **Fuente**: [Hugging Face Model Hub](https://huggingface.co/models).
   - **Ventajas**: Capturan mejor el contexto y significado en oraciones completas.
   - **Desventajas**: Mayor costo computacional comparado con Word2Vec o GloVe.

### 5. **Sentence Transformers (Sentence-BERT)**
   - **Descripción**: Modificación de BERT que permite obtener *embeddings* de frases y párrafos de forma eficiente.
   - **Fuente**: [Sentence Transformers](https://www.sbert.net/).
   - **Ventajas**: Muy útil para tareas de comparación de similitud entre oraciones y recuperación de información.
   - **Desventajas**: Computacionalmente más intensivo que los modelos de solo palabras.

### Comparación de Modelos

| Modelo          | Contexto         | Manejo de OOV (*Out-of-Vocabulary*) | Nivel (Palabra/Frase) | Complejidad Computacional |
|-----------------|------------------|-------------------------------------|-----------------------|----------------------------|
| **Word2Vec**    | Local            | No                                  | Palabra               | Baja                       |
| **GloVe**       | Global           | No                                  | Palabra               | Baja                       |
| **FastText**    | Local con sub-palabras | Sí                             | Palabra               | Media                      |
| **BERT**        | Bidireccional    | Sí (con manejo de contexto)         | Palabra/Frase         | Alta                       |
| **Sentence-BERT** | Bidireccional  | Sí                                  | Frase/Párrafo         | Alta                       |

### Recomendaciones
- **Para aplicaciones ligeras o que necesitan resultados rápidos**: Word2Vec, GloVe o FastText son buenas opciones.
- **Para aplicaciones que requieren alta precisión y comprensión contextual**: BERT, RoBERTa, o Sentence-BERT son más adecuados.
- **Para comparar oraciones o realizar tareas de recuperación de información**: Sentence-BERT es una excelente opción.

## Encontrar un modelo de embedding

---
### TAREA: Busca un tokenizador free
---

In [ ]:
def get_embedding(text: str, model: str = "text-embedding-ada-002"):
    embedding_result = client_openai.embeddings.create(
        model = model,
        input = [text]
    )
    return embedding_result.data[0].embedding

---

In [ ]:
# Tokenizar la entrada
movie_name = movies_ds["Series_Title"].iloc[1]
input_text = movies_ds["Overview"].iloc[1]
movie_name, input_text

## Neo4j INSTANCE

In [ ]:
neo4j_db = Neo4jGraph(
            url = "neo4j+s://01495cb4.databases.neo4j.io",
            username= "neo4j",
            password = ""
        )

### Movies Nodes

In [ ]:
nodes_ = [Node(id = movies_ds["Series_Title"].iloc[it], type = "Movie",
            properties = {
                    "sinopsis": movies_ds["Overview"].iloc[it],
                    "year": movies_ds["Released_Year"].iloc[it],
                    "duration": movies_ds["Runtime"].iloc[it]
                 }) for it in range(movies_ds.shape[0])]

In [ ]:
nodes_[:5]

In [ ]:
nodes = movies_ds.apply(
    lambda row: Node(
        id=row["Series_Title"],
        type="Movie",
        properties={
            "sinopsis": row["Overview"],
            "year": row["Released_Year"],
            "duration": row["Runtime"]
        }
    ), axis=1
).tolist()

In [ ]:
nodes[:5]

# Directors

In [ ]:
# Auxiliary data structures
nodes_directors_aux = {}
nodes_director = []
director_movie_relationships = []

In [ ]:
for row in movies_ds.itertuples():
    director_name = row.Director
    if director_name not in nodes_directors_aux:
        director_node = Node(
            id=director_name,
            type="Director",
            properties={"name": director_name}
        )
        nodes_directors_aux[director_name] = director_node
        nodes_director.append(director_node)
    director_movie_relationships.append(
        Relationship(
            source= nodes[row.Index],
            target= nodes_directors_aux[director_name], # Movie
            type="DIRECTED_BY",
            properties={}
        )
    )

In [ ]:
nodes_director[:5]

In [ ]:
director_movie_relationships[:5]

# Actors

In [ ]:
actors_aux = {}
actors_nodes = []
actor_movie_relationships = []

In [ ]:
for row in movies_ds.itertuples():
    # Verifico y creo nodos para los actores de la película
    for i in range(1, 5):  # Para Star1, Star2, Star3, Star4
        actor_name = getattr(row, f"Star{i}")

        if actor_name and actor_name not in actors_aux:
            # Creo el nodo para el actor si no existe
            actor_node = Node(
                id=actor_name,
                type="Actor",
                properties={"name": actor_name}
            )

            # Guardo el nodo en el diccionario y en la lista
            actors_aux[actor_name] = actor_node
            actors_nodes.append(actor_node)

        # Creo una relación entre la película y el actor
        if actor_name:  # Solo si hay un nombre de actor
            actor_movie_relationships.append(
                Relationship(
                    source=nodes[row.Index],  # Nodo de la película
                    target=actors_aux[actor_name],  # Nodo del actor
                    type="ACTED_IN",
                    properties={}
                )
            )

In [ ]:
actors_nodes[:5]

In [ ]:
actor_movie_relationships[:5]

## Create Graph Documents

### Nodes

In [ ]:
nodes_movies_actors_directors = []
nodes_movies_actors_directors.extend(nodes) # movies
nodes_movies_actors_directors.extend(actors_nodes)
nodes_movies_actors_directors.extend(nodes_director)
len(nodes_movies_actors_directors)

### Relationships

In [ ]:
relationships_movies_actors_directors = []
relationships_movies_actors_directors.extend(director_movie_relationships)
relationships_movies_actors_directors.extend(actor_movie_relationships)
len(relationships_movies_actors_directors)

In [ ]:
doc = Document(page_content="hello world")

In [ ]:
graph_doc = GraphDocument(
    nodes = nodes_movies_actors_directors,
    relationships = relationships_movies_actors_directors,
    source = doc
)

In [ ]:
neo4j_db.add_graph_documents([graph_doc])

In [ ]:
neo4j_db.refresh_schema()

## the END